In [1]:
import sqlite3
import json
import re
import requests
from bs4 import BeautifulSoup

db_path = 'films.db'
json_path = 'films.json'

In [2]:
def getHTMLdocument(url):
    response = requests.get(url)
    return response.content

### Functions for extracting "country" from the url

In [3]:
def extract_alphabetic_substring(text):
    if text[:13] == 'United States':
        return 'United States'
    if text[:14] == 'United Kingdom':
        return 'United Kingdom'
    for i, char in enumerate(text):
        if not char.isalpha() and char != ' ':
            return text[:i]
    return text

def get_country_from_movie_page(movie_url):
    movie_html = getHTMLdocument(movie_url)
    movie_soup = BeautifulSoup(movie_html, 'html.parser')
    infobox = movie_soup.find('table', {'class': 'infobox'})
    if infobox:
        rows = infobox.find_all('tr')
        for row in rows:
            header = row.find('th')
            if header and ('Country' in header.get_text() or 'Countries' in header.get_text()):
                country_td = row.find('td')
                if country_td:
                    return extract_alphabetic_substring(country_td.get_text(strip=True))
    return "Country not found"



### Functions for extracting "directors" from the url

In [4]:
def format_director_name(name):
    return re.sub(r'([a-z])([A-Z])', r'\1 \2', name)

def get_director_from_movie_page(movie_url):
    movie_html = getHTMLdocument(movie_url)
    movie_soup = BeautifulSoup(movie_html, 'html.parser')
    infobox = movie_soup.find('table', {'class': 'infobox'})
    if infobox:
        rows = infobox.find_all('tr')
        for row in rows:
            header = row.find('th')
            if header and 'Directed by' in header.get_text():
                director_td = row.find('td')
                if director_td:
                    return format_director_name(director_td.get_text(strip=True))
    return "Director not found"


### Function to fetch the films and return data with all attributes as a list:

In [5]:
def fetch_films():
    url_to_scrape = "https://en.wikipedia.org/wiki/List_of_highest-grossing_films"
    html_document = getHTMLdocument(url_to_scrape)
    soup = BeautifulSoup(html_document, 'html.parser')
    table = soup.find('table', {'class': 'wikitable'})
    films_data = []
    for row in table.find_all('tr')[1:]:
        movie_title, revenue, year, country, directors = None, None, None, "Unknown", "Unknown"
        cnt = 0
        for i in row.find_all('td'):
            cnt += 1
            if cnt == 3:
                revenue = re.sub(r'[^\d]', '', i.get_text(strip=True))
            if cnt == 4:
                year = i.get_text(strip=True)
        movie_name_tag = row.find('th')
        if movie_name_tag:
            movie_link = movie_name_tag.find('a')
            if movie_link:
                movie_title = movie_link.get_text(strip=True)
                movie_url = "https://en.wikipedia.org" + movie_link.get('href')
                country = get_country_from_movie_page(movie_url)
                directors = get_director_from_movie_page(movie_url)
        if movie_title and revenue and year:
            films_data.append((movie_title, directors, int(year), int(revenue), country))
    return films_data

### Function to take listed data and put to database

In [6]:
def save_to_database(films_data):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    cursor.execute('DROP TABLE IF EXISTS films')
    cursor.execute('''
        CREATE TABLE films (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            title TEXT,
            directors TEXT,
            year INTEGER,
            revenue INTEGER,
            country TEXT
        )
    ''')
    cursor.executemany('INSERT INTO films (title, directors, year, revenue, country) VALUES (?, ?, ?, ?, ?)', films_data)
    conn.commit()
    conn.close()

### Eventially we can get a json data from database:

In [7]:
def fetch_from_database():
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    cursor.execute('SELECT title, directors, year, revenue, country FROM films')
    films_data = cursor.fetchall()
    conn.close()
    return films_data

def save_to_json(films_data):
    films_list = [
        {'title': film[0], 'directors': film[1], 'year': film[2], 'revenue': film[3], 'country': film[4]}
        for film in films_data
    ]
    with open(json_path, 'w', encoding='utf-8') as f:
        json.dump(films_list, f, ensure_ascii=False, indent=4)

def main():
    films_data = fetch_films()
    save_to_database(films_data)

    films_data_from_db = fetch_from_database()
    save_to_json(films_data_from_db)

    print("Films data has been stored in the database and saved to JSON.")

if __name__ == "__main__":
    main()


Films data has been stored in the database and saved to JSON.
